## Creating image patches from hyperspectral images

Due to how large the VIS-NIR hyperspectral image files were, I needed to split the images into smaller chunks or patches to be able to use the images for training a Convolutional Neural Network (CNN). Below is the code for this. In addition PCA is also performed on these patches to reduce the spectral dimensions of the data. 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import IncrementalPCA
import pandas as pd
import numpy as np
import pandas as pd
import os 

In [ ]:
# Read in label files mapping the class labels to filenames
df = pd.read_csv(r"D:\hyperspectral\visnir_patches\labels_day5.csv")
train_df,  test_df = train_test_split(df, test_size=0.2, stratify=df["class"], random_state=42)
train_df.to_csv(r"D:\hyperspectral\visnir_patches\train_day5_labels.csv", index=False)
test_df.to_csv(r"D:\hyperspectral\visnir_patches\test_day5_labels.csv", index=False)

In [ ]:
def open_memmap_cube(path, n_bands, width, dtype=np.float32, mode="r"):
    """
    Function for opening .npy files as a memmap
    
    :param path: path to datafolder
    :param n_bands: number of bands in the data cubes 
    :param width: the width of the data cubes 
    :param dtype: dtype of the data cube 
    :param mode: mode for opening the memmap 
    """
    size_bytes = os.path.getsize(path)
    itemsize = np.dtype(dtype).itemsize
    n_elems = size_bytes // itemsize

    # number of (length * width) elements
    if n_elems % n_bands != 0:
        raise ValueError(
            f"{path}: total elements {n_elems} is not divisible by n_bands={n_bands}"
        )

    n_spatial = n_elems // n_bands

    if n_spatial % width != 0:
        raise ValueError(
            f"{path}: spatial elements {n_spatial} not divisible by width={width}"
        )

    length = n_spatial // width
    shape = (length, width, n_bands)

    return np.memmap(path, dtype=dtype, mode=mode, shape=shape)


In [ ]:
def generate_patches_no_pca(csv_path, data_folder, out_path,
                            expected_bands=673, patch_hw=(25, 25),
                            min_nonzero_ratio=0.5):
    """
    Generate image patches from the hyperspectral image. 
    
    :param csv_path: path to the file mapping labels to filenames
    :param data_folder: path to the data folder 
    :param out_path: path to where the patches are stored 
    :param expected_bands: expected number of bands in the data cubes
    :param patch_hw: patch height and width, given as a tuple (height, width)
    :param min_nonzero_ratio: the minimum number of non-zero pixels for a patch to be saved. 
    """

    df = pd.read_csv(csv_path)
    classes = sorted(df["class"].unique())
    class_to_idx = {c: i for i, c in enumerate(classes)}

    # Only look in these subfolders
    valid_days = ["day2", "day5", "day7", "day9"]

    ph, pw = patch_hw
    patches, labels, image_ids = [], [], []

    for img_idx, row in df.iterrows():

        # Find which folder contains this file
        path = None
        for d in valid_days:
            candidate = os.path.join(data_folder, d, row["filename"])
            if os.path.exists(candidate):
                path = candidate
                break

        if path is None:
            print("File not found in any day folder:", row["filename"])
            continue

        cube = open_memmap_cube(path, n_bands=expected_bands, width=384)
        H, W, B = cube.shape
        if ph > H or pw > W:
            continue

        band0 = cube[..., 0]
        nonzero_mask = band0 != 0

        for top in range(0, H - ph + 1, ph):
            for left in range(0, W - pw + 1, pw):
                m = nonzero_mask[top:top+ph, left:left+pw]
                if m.mean() < min_nonzero_ratio:
                    continue

                patch = cube[top:top+ph, left:left+pw, :]
                patch = np.moveaxis(patch, -1, 0).astype("float32")

                patches.append(patch)
                labels.append(class_to_idx[row["class"]])
                image_ids.append(img_idx)

    patches = np.stack(patches, 0)
    labels = np.asarray(labels, dtype=np.int64)
    image_ids = np.asarray(image_ids, dtype=np.int32)

    np.savez_compressed(out_path,
                        patches=patches,
                        labels=labels,
                        image_ids=image_ids,
                        patch_hw=patch_hw,
                        expected_bands=expected_bands)

    print("Saved:", out_path, patches.shape, labels.shape, image_ids.shape)


In [ ]:
# ONLY RUN ONCE
generate_patches_no_pca(csv_path=r"D:\hyperspectral\visnir_patches\train_day5_labels.csv", data_folder=r"D:\hyperspectral\visnir\masked_images_day5", out_path=r"D:\hyperspectral\visnir_masked\train_patches_day5_ids.npz",  patch_hw=(25, 25), expected_bands=673)
generate_patches_no_pca(csv_path=r"D:\hyperspectral\visnir_patches\test_day5_labels.csv", data_folder=r"D:\hyperspectral\visnir\masked_images_day5",out_path=r"D:\hyperspectral\visnir_masked\test_patches_day5_ids.npz", patch_hw=(25, 25), expected_bands=673)

Saved: D:\hyperspectral\swir_patches\train_patches_day9_swir.npz (339, 288, 32, 32) (339,) (339,)
Saved: D:\hyperspectral\swir_patches\test_patches_day9_swir.npz (79, 288, 32, 32) (79,) (79,)


In [ ]:

def fit_pca_on_patches(npz_path, n_components=50, batch_pixels=200_000):
    """
    Fits PCA on the image patches
    
    :param npz_path: path to the data folder with the image patches
    :param n_components: number of principal components 
    :param batch_pixels: number of pixels to base the PCA on
    """
    data = np.load(npz_path)
    patches = data["patches"]           # (N, B, ph, pw)
    N, B, ph, pw = patches.shape

    ipca = IncrementalPCA(n_components=n_components)

    # iterate in chunks of patches to avoid RAM blowup
    idx = 0
    while idx < N:
        end = min(idx + 64, N)         # 64 patches at a time, tune as needed
        batch = patches[idx:end]       # (batch, B, ph, pw)
        batch_flat = batch.reshape(-1, B)  # pixels: (batch*ph*pw, B)

        # to reduce compute, you can randomly subsample rows from batch_flat
        if batch_flat.shape[0] > batch_pixels:
            choice = np.random.choice(batch_flat.shape[0],
                                      size=batch_pixels, replace=False)
            batch_flat = batch_flat[choice]

        ipca.partial_fit(batch_flat)
        idx = end

    return ipca


In [ ]:
def apply_pca_to_patches(npz_in, npz_out, pca):
    """
    Applies PCA to the image patches. 
    
    :param npz_in: path to the data folder with the image patches
    :param npz_out: path to the folder where the PCA patches is to be stored
    :param pca: pca function 
    """
    i
    data = np.load(npz_in)
    P = data["patches"]        # (N, B, ph, pw)
    y = data["labels"]         # (N,)
    img_ids = data["image_ids"]# (N,)
    N, B, ph, pw = P.shape
    K = pca.n_components_

    Pp = np.empty((N, K, ph, pw), dtype=np.float32)
    for i in range(N):
        p = P[i].reshape(B, -1).T               # (ph*pw, B)
        Pp[i] = pca.transform(p).astype(np.float32).T.reshape(K, ph, pw)

    np.savez_compressed(npz_out,
                        patches=Pp,
                        labels=y,
                        image_ids=img_ids,      # <-- keep ids
                        patch_hw=(ph, pw),
                        n_components=K)
    print("Saved PCA patches:", npz_out, Pp.shape, y.shape, img_ids.shape)


In [38]:
pca = fit_pca_on_patches(npz_path=r"D:\hyperspectral\visnir_masked\train_patches_day5_ids.npz", n_components=32)
apply_pca_to_patches(npz_in=r"D:\hyperspectral\visnir_masked\train_patches_day5_ids.npz", npz_out=r"D:\hyperspectral\visnir_masked\pca_train_patches_day5_32.npz", pca=pca)
apply_pca_to_patches(npz_in=r"D:\hyperspectral\visnir_masked\test_patches_day5_ids.npz", npz_out=r"D:\hyperspectral\visnir_masked\pca_test_patches_day5_32.npz", pca=pca)


Saved PCA patches: D:\hyperspectral\visnir_masked\pca_train_patches_day5_32.npz (9601, 32, 25, 25) (9601,) (9601,)
Saved PCA patches: D:\hyperspectral\visnir_masked\pca_test_patches_day5_32.npz (2754, 32, 25, 25) (2754,) (2754,)


In [39]:
pca = fit_pca_on_patches(npz_path=r"D:\hyperspectral\visnir_masked\train_patches_day5_ids.npz", n_components=16)
apply_pca_to_patches(npz_in=r"D:\hyperspectral\visnir_masked\train_patches_day5_ids.npz", npz_out=r"D:\hyperspectral\visnir_masked\pca_train_patches_day5_16.npz", pca=pca)
apply_pca_to_patches(npz_in=r"D:\hyperspectral\visnir_masked\test_patches_day5_ids.npz", npz_out=r"D:\hyperspectral\visnir_masked\pca_test_patches_day5_16.npz", pca=pca)

Saved PCA patches: D:\hyperspectral\visnir_masked\pca_train_patches_day5_16.npz (9601, 16, 25, 25) (9601,) (9601,)
Saved PCA patches: D:\hyperspectral\visnir_masked\pca_test_patches_day5_16.npz (2754, 16, 25, 25) (2754,) (2754,)
